In [ ]:
##############Assignment 5#################
###########################################


#################Part 1#################
#Parts of the code in this assignment were adapted from the lecture notes.


import numpy as np
import camb
from matplotlib import pyplot as plt
import time

#########Following is based on planck_likelihood.py.

def get_spectrum(pars,lmax=3000):
    #print('pars are ',pars)
    H0=pars[0]
    ombh2=pars[1]
    omch2=pars[2]
    tau=pars[3]
    As=pars[4]
    ns=pars[5]
    pars=camb.CAMBparams()
    pars.set_cosmology(H0=H0,ombh2=ombh2,omch2=omch2,mnu=0.06,omk=0,tau=tau)
    pars.InitPower.set_params(As=As,ns=ns,r=0)
    pars.set_for_lmax(lmax,lens_potential_accuracy=0)
    results=camb.get_results(pars)
    powers=results.get_cmb_power_spectra(pars,CMB_unit='muK')
    cmb=powers['total']
    tt=cmb[:,0]    #you could return the full power spectrum here if you wanted to do say EE
    return tt[2:][:2507]


plt.ion()

pars=np.asarray([60,0.02,0.1,0.05,2.00e-9,1.0])
planck=np.loadtxt('COM_PowerSpect_CMB-TT-full_R3.01.txt',skiprows=1)
ell=planck[:,0]
spec=planck[:,1]
errs=0.5*(planck[:,2]+planck[:,3]);
model=get_spectrum(pars)
model=model[:len(spec)]
resid=spec-model
chisq=np.sum( (resid/errs)**2)
print("chisq is ",chisq," for ",len(resid)-len(pars)," degrees of freedom.")
#read in a binned version of the Planck PS for plotting purposes
planck_binned=np.loadtxt('COM_PowerSpect_CMB-TT-binned_R3.01.txt',skiprows=1)
errs_binned=0.5*(planck_binned[:,2]+planck_binned[:,3]);
plt.clf()
plt.plot(ell,model, label='Model')
plt.errorbar(planck_binned[:,0],planck_binned[:,1],errs_binned,fmt='.', label='Data')
plt.xlabel('Multipole')
plt.ylabel('Spectrum')
plt.title('Spectrum Profile with Initial Parameters')
plt.legend()
plt.show()


#########



In [ ]:
#########Chi-squared analysis with parameters of [69, 0.022, 0.12, 0.06, 2.1e-9, 0.95]
pars=np.asarray([69, 0.022, 0.12, 0.06, 2.1e-9, 0.95])
model=get_spectrum(pars)
model=model[:len(spec)]
resid=spec-model
chisq=np.sum( (resid/errs)**2)
print("chisq is ",chisq," for ",len(resid)-len(pars)," degrees of freedom.")
#read in a binned version of the Planck PS for plotting purposes
planck_binned=np.loadtxt('COM_PowerSpect_CMB-TT-binned_R3.01.txt',skiprows=1)
errs_binned=0.5*(planck_binned[:,2]+planck_binned[:,3]);
plt.clf()
plt.plot(ell,model,label='Model')
plt.errorbar(planck_binned[:,0],planck_binned[:,1],errs_binned,fmt='.',label='Data')
plt.xlabel('Multipole')
plt.ylabel('Spectrum')
plt.title('Spectrum Profile with Revised Parameters')
plt.legend()
plt.show()
#########
#Degrees of freedom basic statistics
dof=2501 
vardof=2*dof
stdevdof=np.sqrt(vardof)
print(stdevdof)
print("Reference chiqsq mean=" + str(dof) + ", variance=" +str(vardof)+", stdev="+str(stdevdof))
######### 


In [ ]:
#################Part 2#################

#Newton's method to calculate best fit parameters
def fitnewton(fun, pars, dp, spec, ell):
    iter_no=20;#specify number of iterations for fit
    ellen=len(ell)
    parlen=len(pars)
    ainv=np.diag(1/errs**2)

    for i in range(iter_no):
        mspec=fun(pars)
        M=np.empty([ellen,parlen])
        for i in range(len(pars)):
            p1=pars.copy()
            p2=pars.copy()
            p1[i]=pars[i]-dp[i]
            p2[i]=pars[i]+dp[i]
            y1=fun(p1)
            y2=fun(p2)
            M[:,i]=(y2-y1)/(2*dp[i])
            
        res=spec-mspec
        lhs=np.linalg.inv(M.T@ainv@M)
        rhs=M.T@ainv@res
        dp=lhs@rhs
        pars=pars+dp
        print(dp)

    perr=np.sqrt((np.diag(lhs))) # parameter errors
    return pars, perr, lhs

pars3=pars # Starting point of estimation [69, 0.022, 0.12, 0.06, 2.1e-9, 0.95]

dp=np.array([1e-1, 1e-4, 1e-3, 1e-3, 1e-11, 1e-2]) # step size ~1% of pars

parfit=fitnewton(get_spectrum, pars3, dp, spec, ell) #perform Newton's fit

print('Parameters ', parfit[0])
print('Errors ', parfit[1])


In [ ]:
model2=get_spectrum(parfit[0])
resid=spec-model2
chisq=np.sum( (resid/errs)**2)
print("chisq is ",chisq," for ",len(resid)-len(parfit[0])," degrees of freedom.")
plt.clf()
plt.plot(ell,model2, label='Model')
plt.errorbar(planck_binned[:,0],planck_binned[:,1],errs_binned,fmt='.', label='Data')
plt.xlabel('Multipole')
plt.ylabel('Spectrum')
plt.legend()
plt.title('Spectrum Profile - Newton\'s Fit')
#save results
#Parameter names

paranamesave= ['Ho','Obh2','Och2', 'T', 'As', 'ns']
pfile='planck_fit_params.txt'
np.savetxt(pfile,np.column_stack([paranamesave,parfit[0],parfit[1]]),fmt='%s')

In [ ]:
#Part 2 parameters and errors
for i in range(len(parfit[0])):
    print (paranamesave[i]+ ' = ' +format(parfit[0][i])+' \u00B1 ' + format(parfit[1][i]))

In [ ]:
#################Part 3#################

#MCMC functions
def lor_chisq(pars,x,y,noise=None):
    pred=get_spectrum(pars)
    if noise is None:
        return np.sum((y-pred)**2)
    else:
        return np.sum (((y-pred)/noise)**2)

def mcmc(pars,step_size,x,y,fun,nstep=10,noise=errs):
    chi_cur=fun(pars,x,y,noise)
    npar=len(pars)
    chain=np.zeros([nstep,npar])
    chivec=np.zeros(nstep)
    for i in range(nstep):
        trial_pars=pars+np.random.multivariate_normal(np.zeros(len(pars)), step_size)
        trial_chisq=fun(trial_pars,x,y,noise)
        delta_chisq=trial_chisq-chi_cur
        accept_prob=np.exp(-0.5*delta_chisq)
        accept=np.random.rand(1)<accept_prob
        if accept:
            pars=trial_pars
            chi_cur=trial_chisq
        chain[i,:]=pars
        chivec[i]=chi_cur
    return chain,chivec
#from part 2 best fit
# pars3 = parfit[0].copy()
parstart=np.array([69, 0.022, 0.12, 0.06, 2.1e-9, 0.95]) 
step_size=parfit[2].copy()


chain3p,chisq3p=mcmc(parstart,step_size,ell,spec,lor_chisq,nstep=5000)




In [ ]:
#Part3 save results
pfile='planck_chain.txt'
np.savetxt(pfile,np.column_stack([chisq3p,chain3p]),fmt='%s')

In [ ]:
#Dark Energy calculations
#remove first 1000 chain data
meanparsd=np.mean(chain3p[1000:], axis=0)#mean parameters
errparsd=np.std(chain3p[1000:], axis=0)#stdev parameters
h3=meanparsd[0]/100
darken=1-(meanparsd[1]/h3**2+meanparsd[2]/h3**2)
darkenerr=np.sqrt((1e4*meanparsd[1]/(meanparsd[0]**2))**2*((errparsd[1]/meanparsd[1])**2 +(2*errparsd[0]/meanparsd[0])**2)+
(1e4*meanparsd[2]/(meanparsd[0]**2))**2*((errparsd[2]/meanparsd[2])**2 +(2*errparsd[0]/meanparsd[0])**2))
darkenerr
print('Dark Energy = ' +format(darken) + ' +/- ' + format(darkenerr))



In [ ]:
#Part 3 MCMC parameter plots

#Parameter names
paranames = ['$H_0$','$\u03A9_b$$h^2$','$\u03A9_c$$h^2$', '\u03C4', '$A_s$', '$n_s$']

for i in range(len(pars)):
    plt.subplot(3,2,i+1)
    plt.plot(chain3p[:,i])
    plt.xlabel('Chain Index')
    plt.ylabel('{}'.format(paranames[i]))
#     plt.legend()
#     plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.tight_layout(pad=1.0)
# plt.title('MCMC - Parameters')
plt.show()

In [ ]:
#Part 3 frequency domain plots
for i in range(len(pars)):
    plt.subplot(3,2,i+1)
    plt.loglog(np.abs(np.fft.rfft(chain3p[:,i])))
    plt.xlabel('Frequency')
    plt.ylabel('Power {}'.format(paranames[i]))
#     plt.legend()
#     plt.title('MCMC - parameter')
#     plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
#Part 3 plot chisq 
plt.clf()
plt.plot(chisq3p)
plt.xlabel('Chain Index')
plt.ylabel('$\u03c7^2$')
plt.show()

In [ ]:
model3=get_spectrum(meanparsd)
resid3=spec-model3
chisq3=np.sum( (resid3/errs)**2)
print("chisq is ",chisq3," for ",len(resid)-len(parfit[0])," degrees of freedom.")
plt.clf()
plt.plot(ell,model3, label='Model')
plt.errorbar(planck_binned[:,0],planck_binned[:,1],errs_binned,fmt='.', label='Data')
plt.xlabel('Multipole')
plt.ylabel('Spectrum')
plt.title('Spectrum Profile - MCMC Part 3')
plt.legend()
plt.show()
#save results
#Parameter names
# paranames = ['Ho','Obh2','Och2', 'T', 'As', 'ns']
# pfile='planck_fit_params.txt'
# np.savetxt(pfile,np.column_stack([paranames,parfit[0],parfit[1]]),fmt='%s')

In [ ]:
#Part 3 parameters and errors
for i in range(len(meanparsd)):
    print (paranamesave[i]+ ' = ' +format(meanparsd[i])+' \u00B1 ' + format(errparsd[i]))

In [ ]:
#################Part 4#################

#calculate additional term to add to chisq
def dchisq(T):
    meanT=0.0540 
    errT=0.0074
    dchi=((T-meanT)/errT)**2
    return dchi

#calculate weighting factor for new covariance calculation
w=np.exp(-0.5*dchisq(chain3p[:,3]))

#new covariance matrix based on the weight
newcov = np.cov(chain3p.T, aweights=w)

#modified mcmc with T constraint and additional chisq term
def mcmcT(pars,step_size,x,y,fun,nstep=10,noise=errs):
    chi_cur=fun(pars,x,y,noise)
    npar=len(pars)
    chain=np.zeros([nstep,npar])
    chivec=np.zeros(nstep)
    for i in range(nstep):
 #       print(i)
#         trial_pars=pars+step_size*np.random.randn(npar)
        trial_pars=pars+np.random.multivariate_normal(np.zeros(len(pars)), step_size)
        trial_chisq=fun(trial_pars,x,y,noise)+dchisq(trial_pars[3])# add the additional chisq term
        delta_chisq=trial_chisq-chi_cur
        accept_prob=np.exp(-0.5*delta_chisq)
        accept=np.random.rand(1)<accept_prob
        if accept:
            pars=trial_pars
            chi_cur=trial_chisq
        chain[i,:]=pars
        chivec[i]=chi_cur
    return chain,chivec


parstart4=np.array([69, 0.022, 0.12, 0.054, 2.1e-9, 0.95])#starting parameters accounting for T=0.054
step_size4=newcov


chain4,chisq4p=mcmcT(parstart4,step_size4,ell,spec,lor_chisq,nstep=5000)

In [ ]:
#Part4 save results
pfile='planck_chain_tauprior.txt'
np.savetxt(pfile,np.column_stack([chisq4p,chain4]),fmt='%s')

In [ ]:
#Part 4 MCMC parameter plots
for i in range(len(pars)):
    plt.subplot(3,2,i+1)
    plt.plot(chain4[:,i])
    plt.xlabel('Chain Index')
    plt.ylabel('Parameter {}'.format(paranames[i]))
#     plt.legend()
#     plt.title('MCMC - parameter')
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
#Part 4 frequency domain plots
for i in range(len(pars)):
    plt.subplot(3,2,i+1)
    plt.loglog(np.abs(np.fft.rfft(chain4[:,i])))
    plt.xlabel('Chain Index')
    plt.ylabel('Power {}'.format(paranames[i]))
#     plt.legend()
#     plt.title('MCMC - parameter')
#     plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
#Part 4 plot chisq
plt.clf()
plt.plot(chisq4p)
plt.xlabel('Chain Index')
plt.ylabel('$\u03c7^2$')
plt.show()

In [ ]:
#Part 4 resultant parameters
meanparsd4=np.mean(chain4[10:], axis=0)#mean parameters
errparsd4=np.std(chain4[10:], axis=0)#stdev parameters
model4=get_spectrum(meanparsd4)
resid4=spec-model4
chisq4=np.sum( (resid4/errs)**2)
print("chisq is ",chisq4," for ",len(resid)-len(parfit[0])," degrees of freedom.")
plt.clf()
plt.plot(ell,model4)
plt.errorbar(planck_binned[:,0],planck_binned[:,1],errs_binned,fmt='.')
plt.xlabel('Multipole')
plt.ylabel('Spectrum')
plt.title('Spectrum Profile - MCMC Part 4')
plt.show()


In [ ]:
#Part 4 T constrained parameters and errors
for i in range(len(meanparsd4)):
    print (paranamesave[i]+ ' = ' +format(meanparsd4[i])+' \u00B1 ' + format(errparsd4[i]))

In [ ]:
import corner
#Part 3 corner plots

plt.ioff()
corner.corner(chain3p, labels=paranames, show_titles=True)

In [ ]:
#Part 4 corner plots
figure=corner.corner(chain4)

plt.ioff()
corner.corner(chain4, labels=paranames, show_titles=True)

In [ ]:
#part4 Bonus 2
bounds=np.array([meanparsd4-5*errparsd4, meanparsd4+5*errparsd4])
for i in range(len(bounds[0])):
    print(format(bounds[0][i])+','+format(bounds[1][i]))

In [ ]:
#importance sampling
#function to calculate weighting actor
def wcalc(Tref):
    w = np.exp(-0.5*dchisq(Tref))
    return w

#weighting factor for T
weight3=wcalc(chain3p[:,3])

#arrays to store importance sampling new parameters and errors
weightpars=np.zeros(len(meanparsd))
weightcovnew =np.zeros((len(meanparsd), len(meanparsd)))

#generate the new weighted parameter based on importance sampling and find the errors
for k in range(len(meanparsd)):
    weightpars[k]=np.average(chain3p[:,k], weights=weight3)#weighted average of new parameters
for i in range(len(meanparsd)):
    for j in range(len(meanparsd)):
        weightcovnew[i,j]=np.average(chain3p[:,i]*chain3p[:,j],weights=weight3)- weightpars[i]*weightpars[j]

weighterrs=np.sqrt(np.diag(weightcovnew))


In [ ]:
print (weightpars)

In [ ]:
print(weighterrs)

In [ ]:
#importance sampling parameter and errors
for i in range(len(weightpars)):
    print (paranamesave[i]+ ' = ' +format(weightpars[i])+' \u00B1 ' + format(weighterrs[i]))